In [7]:
import importlib

import RktnChallenge.RktnModel
importlib.reload(RktnChallenge.RktnModel)
from RktnChallenge.RktnModel import ModelTrainer

import RktnChallenge.preprocessing.translater
importlib.reload(RktnChallenge.preprocessing.translater)
from RktnChallenge.preprocessing.translater import LanguageDetector
from RktnChallenge.preprocessing.translater import Translator

import RktnChallenge.preprocessing.CopyWhereNan
importlib.reload(RktnChallenge.preprocessing.CopyWhereNan)
from RktnChallenge.preprocessing.CopyWhereNan import CopyWhereNan

import RktnChallenge.preprocessing.removeHTML
importlib.reload(RktnChallenge.preprocessing.removeHTML)
from RktnChallenge.preprocessing.removeHTML import removeHTML

On charge la base et init le support GPU / CPU

In [8]:
RktnModel = ModelTrainer("data.csv")

# On initialise le GPU disponible, la gestion mémoire
RktnModel.initGPU()

1 Physical GPUs, 1 Logical GPUs


On détecte la langue de la source et range dans une nouvelle colonne

In [9]:
desc_removeHTML = removeHTML("description")
desi_removeHTML = removeHTML("designation")

desi_langueDetect = LanguageDetector("desi_langue","designation",confiance=0.8)
desc_langueDetect = LanguageDetector("desc_langue","description",confiance=0.8)


desi_Translator = Translator(dest = "tr_designation", src="designation",               
                             detected_lang="desi_langue", source = "google",verbose = False)

desc_Translator = Translator(dest = "tr_description", src="description",               
                             detected_lang="desc_langue", source = "google", verbose = False)



RktnModel.initPreprocess()
# unidecode et removeHTML
RktnModel.addPreprocessStep("desc removeHTML",desc_removeHTML)
RktnModel.addPreprocessStep("desi removeHTML",desi_removeHTML)
# decode la langue et traduit la designation
RktnModel.addPreprocessStep("detect langue designation",desi_langueDetect)
RktnModel.addPreprocessStep("translate designation",desi_Translator)
# decode la langue et traduit la description
RktnModel.addPreprocessStep("detect langue description",desc_langueDetect)
RktnModel.addPreprocessStep("translate description",desc_Translator)
RktnModel.preprocess()


print("langues detectees dans les designations: ",desi_langueDetect.langues)
print("langues detectees dans les descriptions: ",desc_langueDetect.langues)

/home/pec/Rktn/RktnChallenge/preprocessing/removeHTML.py:17: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  X[self.dest] = X[self.dest].apply(lambda x: "" if(pd.isna(x)) else unidecode(BeautifulSoup(x, "lxml").text))
/home/pec/Rktn/RktnChallenge/preprocessing/removeHTML.py:17: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  X[self.dest] = X[self.dest].apply(lambda x: "" if(pd.isna(x)) else unidecode(BeautifulSoup(x, "lxml").text))
/home/pec/Rktn/RktnChallenge/preprocessing/removeHTML.py:17: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  X[self.dest] = X[self.dest].apply(lambda x: "" if(pd.isna(x)) else unidecode

Loading lid Lang model


fin de detection
translating ... designation
erreur:  Hallwag Carte Routiere Deutschland 
erreur:  Totika Set Five Card Decks Included
erreur:  S.I.C. Kamen Rider Zx (Completed) Bandai [Japan]  7""
erreur:  Jahrbuch Der Luftwaffe / Folge 12
erreur:  Guitar Hero 2 Bundle With Guitar
erreur:  Coprirete In Feltro Cod.020-7100
erreur:  Dr. Whostealth Cyberman 6th. Doctor
erreur:  VijverTechniek (VT) Filtre de bassin Vex-200
erreur:  Leitz 50 X Pendelhefter Orgacolor A4 Kaufmannische Heftung
fin de translation
fin de detection
translating ... description
erreur:  p7 inch plush features Hello Kitty dressed as an Ugly Doll.p
erreur:  pEquipment and furniture for playing in the kitchen of Amandas house. Made of farmed wood. Includes stove with oven sink with cupboard table with two stools coffee machine and toaster. Furnish the entire house and collect all the members of this playful family. Wood from repopulated forests. For each tree felled three more are planted. Amanda and her troupe are a

In [11]:
display(RktnModel.data.tail(20))

,Unnamed: 0,designation,description,productid,imageid,prdtypecode,imgname,desi_langue,tr_designation,desc_langue,tr_description
84896,84896,Elder Law For Paralegals,,181592876,990135434,10,image_990135434_product_181592876.jpg,fr,NaN,fr,NaN
84897,84897,Gamme Pro: Gobelets Predoses A Opercule Chocol...,poids net: 360g,2565819932,1149850954,1940,image_1149850954_product_2565819932.jpg,fr,NaN,fr,NaN
84898,84898,Patrie Suisse (La) Ndeg 136 Du 07/12/1898 - Al...,,570634446,1027296807,2280,image_1027296807_product_570634446.jpg,fr,NaN,fr,NaN
84899,84899,Fallout 76 Tricentennial Edition - Jeu En Tele...,NOTE : Un compte Bethesda et une connexion in...,3850278820,1252659479,2905,image_1252659479_product_3850278820.jpg,fr,NaN,fr,NaN
84900,84900,Mi Libro De Cortar Y Pegar,,85849076,895279629,10,image_895279629_product_85849076.jpg,fr,NaN,fr,NaN
84901,84901,"Figurine ""Bip / Viacom"" - Dora L'exploratrice ...","figurines ""Viacom"" (surprises Daniel Roche) co...",421484600,1006234772,1140,image_1006234772_product_421484600.jpg,fr,NaN,fr,NaN
84902,84902,Lit En Tissu Double 180x200 Cm Beige Ambassador,Superbe lit double au style intemporelCe super...,4170588152,1307103543,1560,image_1307103543_product_4170588152.jpg,fr,NaN,fr,NaN
84903,84903,73pcs Block Set Rotation Bricolage Twister Car...,73pcs Block Set Rotation Bricolage Twister Car...,4006488781,1275489688,1281,image_1275489688_product_4006488781.jpg,fr,NaN,fr,NaN
84904,84904,Coller Porte-Brosse A Dents Brosse A Dents Por...,Coller Porte-brosse a dents Brosse a dents por...,4179411158,1309066528,1560,image_1309066528_product_4179411158.jpg,fr,NaN,fr,NaN
84905,84905,Bache ete pour piscine Lili O 255 m,La bache ete permet d'augmenter et de mainteni...,1886035156,1109239046,2583,image_1109239046_product_1886035156.jpg,fr,NaN,fr,NaN


In [12]:
# on copie les lignes qui n'ont pas été traduites (celles en français)
desc_copy = CopyWhereNan("tr_description","description")
desi_copy = CopyWhereNan("tr_designation","designation")

RktnModel.initPreprocess()
RktnModel.addPreprocessStep("desc copy french",desc_copy)
RktnModel.addPreprocessStep("desi copy french",desi_copy)
RktnModel.data = RktnModel.preprocess()

#rajouter le drop de la detection de langue


RktnModel.saveCSV("data_tr.csv")


In [14]:
display(RktnModel.data)

,Unnamed: 0,designation,description,productid,imageid,prdtypecode,imgname,desi_langue,tr_designation,desc_langue,tr_description
0,0,Olivia: Personalisiertes Notizbuch / 150 Seite...,,3804725264,1263597046,10,image_1263597046_product_3804725264.jpg,fr,Olivia: Personalisiertes Notizbuch / 150 Seite...,fr,
1,1,Journal Des Arts (Le) Ndeg 133 Du 28/09/2001 -...,,436067568,1008141237,2280,image_1008141237_product_436067568.jpg,fr,Journal Des Arts (Le) Ndeg 133 Du 28/09/2001 -...,fr,
2,2,Grand Stylet Ergonomique Bleu Gamepad Nintendo...,PILOT STYLE Touch Pen de marque Speedlink est ...,201115110,938777978,50,image_938777978_product_201115110.jpg,fr,Grand Stylet Ergonomique Bleu Gamepad Nintendo...,fr,PILOT STYLE Touch Pen de marque Speedlink est ...
3,3,Peluche Donald - Europe - Disneyland 2000 (Mar...,,50418756,457047496,1280,image_457047496_product_50418756.jpg,fr,Peluche Donald - Europe - Disneyland 2000 (Mar...,fr,
4,4,La Guerre Des Tuques,Luc a des idees de grandeur. Il veut organiser...,278535884,1077757786,2705,image_1077757786_product_278535884.jpg,fr,La Guerre Des Tuques,fr,Luc a des idees de grandeur. Il veut organiser...
...,...,...,...,...,...,...,...,...,...,...,...
84911,84911,The Sims [ Import Anglais ],,206719094,941495734,40,image_941495734_product_206719094.jpg,fr,The Sims [ Import Anglais ],fr,
84912,84912,Kit piscine acier NEVADA deco pierre O 3.50m x...,Description complete :Kit piscine hors-sol Toi...,3065095706,1188462883,2583,image_1188462883_product_3065095706.jpg,fr,Kit piscine acier NEVADA deco pierre O 3.50m x...,fr,Description complete :Kit piscine hors-sol Toi...
84913,84913,Journal Officiel De La Republique Francaise Nd...,,440707564,1009325617,2280,image_1009325617_product_440707564.jpg,fr,Journal Officiel De La Republique Francaise Nd...,fr,
84914,84914,Table Basse Bois De Recuperation Massif Base B...,Cette table basse a un design unique et consti...,3942400296,1267353403,1560,image_1267353403_product_3942400296.jpg,fr,Table Basse Bois De Recuperation Massif Base B...,fr,Cette table basse a un design unique et consti...
